https://pyy0715.github.io/2020/03/14/Albumentation_Tutorial/

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset

import albumentations

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
class_map = pd.read_csv("../input/bengaliai-cv19/class_map.csv")
sample_submission = pd.read_csv("../input/bengaliai-cv19/sample_submission.csv")
test = pd.read_csv("../input/bengaliai-cv19/test.csv")
train = pd.read_csv("../input/bengaliai-cv19/train.csv")

In [ ]:
data_dir = '/kaggle/input/bengaliai-cv19/'
files_train = [f'train_image_data_{fid}.parquet' for fid in range(4)]

In [ ]:
F = os.path.join(data_dir, files_train[0])

train0 = pd.read_parquet(F)

In [ ]:
train0.head()

In [ ]:
print('Train Image Data{} Shape is : {}'.format(0, train0.shape))

In [ ]:
class BengaliDataset(Dataset):
    def __init__(self, df, img_height, img_width):
        self.df = df
        self.img_height = img_height
        self.img_width = img_width

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = self.df.iloc[idx][0:].values.astype(np.uint8)
        img = img.reshape(self.img_height, self.img_width)
        img = 255 - img
        img = (img*(255.0/img.max())).astype(np.uint8)

        return img

In [ ]:
HEIGHT = 137
WIDTH = 236

train0.set_index('image_id', inplace=True)
image = BengaliDataset(train0, img_height=HEIGHT, img_width=WIDTH)

In [ ]:
nrow, ncol = 1, 5

fig, axs = plt.subplots(nrow, ncol, figsize=(20, 10))
axs = axs.flatten()

for i, ax in enumerate(axs):
    img = image[i]
    ax.imshow(img)
    ax.set_title(f'label: Original')
    ax.axis('off')
plt.tight_layout()

In [ ]:
import albumentations as A

aug = A.IAAPiecewiseAffine(p=1.0)

nrow, ncol = 5, 2
fig, axs = plt.subplots(nrow, ncol, figsize=(20, 10))

for n in range(nrow):
    img = image[n]
    aug_image = aug(image=img)['image']
    
    axs[n,0].imshow(img)
    axs[n,0].set_title(f'label: Original')
    axs[n,0].axis('off')
    
    axs[n,1].imshow(aug_image)
    axs[n,1].set_title(f'label: Affine')
    axs[n,1].axis('off')
    
plt.tight_layout()

AugMix
Alubumentiation을 기반으로 AugMix가 어떤것인지 알아보겠습니다.

먼저 아래의 사진 4장을 보면서, 대략적인 감을 잡으실 수 있을겁니다.

CutOut - 이미지의 임의의 부분을 제거

MixUp - 이미지 간 확률적으로 두 이미지를 섞습니다.

CutMix - CutOut + MixUp

AugMix - 위에서 소개한 Augmentation기법들을 섞습니다.

AugMix는 이름처럼 여러 data augmentation 방법들을 혼합하여 사용한다.

하지만 기존 방법들과 달리 데이터 변환으로인한 manifold를 벗어나는 일을 방지하였다.


![](https://storage.googleapis.com/groundai-web-prod/media/users/user_135639/project_400799/images/x1.png)

![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F448347%2Fe9d3259d5b0ef3dadba0238bf3901f1e%2F2020-02-10%2013.50.53.png?generation=1581310282118474&alt=media)

![](https://github.com/google-research/augmix/raw/master/assets/augmix.gif)


CIFAR-10 실험결과, AugMix는 다른 기법들보다 우수하게 성적을 냈다고 합니다.

Augmix: https://arxiv.org/abs/1912.02781

AugMix: A Simple Data Processing Method to Improve Robustness and Uncertainty
https://arxiv.org/pdf/1912.02781v2.pdf


Official implementation: https://github.com/google-research/augmix

In [ ]:
from albumentations import HorizontalFlip
from albumentations.core.transforms_interface import ImageOnlyTransform

class AugMix(ImageOnlyTransform):
    """Augmentations mix to Improve Robustness and Uncertainty.
    Args:
        image (np.ndarray): Raw input image of shape (h, w, c)
        severity (int): Severity of underlying augmentation operators.
        width (int): Width of augmentation chain
        depth (int): Depth of augmentation chain. -1 enables stochastic depth uniformly
          from [1, 3]
        alpha (float): Probability coefficient for Beta and Dirichlet distributions.
        augmentations (list of augmentations): Augmentations that need to mix and perform.
    Targets:
        image
    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/1912.02781
    |  https://github.com/google-research/augmix
    """

    def __init__(self, width=2, depth=2, alpha=0.5, augmentations=[HorizontalFlip()], always_apply=False, p=0.5):
        super(AugMix, self).__init__(always_apply, p)
        self.width = width
        self.depth = depth
        self.alpha = alpha
        self.augmentations = augmentations
        self.ws = np.float32(np.random.dirichlet([self.alpha] * self.width))
        self.m = np.float32(np.random.beta(self.alpha, self.alpha))

    def apply_op(self, image, op):
        image = op(image=image)["image"]
        return image

    def apply(self, img, **params):
        mix = np.zeros_like(img)
        for i in range(self.width):
            image_aug = img.copy()

            for _ in range(self.depth):
                op = np.random.choice(self.augmentations)
                image_aug = self.apply_op(image_aug, op)

            mix = np.add(mix, self.ws[i] * image_aug, out=mix, casting="unsafe")

        mixed = (1 - self.m) * img + self.m * mix
        if img.dtype in ["uint8", "uint16", "uint32", "uint64"]:
            mixed = np.clip((mixed), 0, 255).astype(np.uint8)
        return mixed

    def get_transform_init_args_names(self):
        return ("width", "depth", "alpha")

AugMix는 아래와 같이 여러가지 기법들을 동시에 적용할 수 있으며, 확률값으로 조정이 가능합니다.

또한 OneOf를 사용하여, 기법 중에서 하나의 기법만 선택할 수 있습니다.

In [ ]:
augs = [A.HorizontalFlip(always_apply=True),
        A.Blur(always_apply=True),
        A.ShiftScaleRotate(always_apply=True),
        A.GaussNoise(always_apply=True),
        A.Cutout(always_apply=True),
        A.IAAPiecewiseAffine(always_apply=True)]

transforms_train = albumentations.Compose([
    AugMix(width=3, depth=2, alpha=.4, p=1., augmentations=augs),
])

In [ ]:
class Aug_BengaliDataset(Dataset):
    def __init__(self, df, img_height, img_width, transform=None):
        self.df = df
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform

    def __len__(self):
        return len(self.df)
    

    def __getitem__(self, idx):
        img = self.df.iloc[idx][0:].values.astype(np.uint8)
        img = img.reshape(self.img_height, self.img_width)
        img = 255 - img
        img = (img*(255.0/img.max())).astype(np.uint8)
        
        if self.transform is not None:
            res = self.transform(image=img)
            img = res['image']
        else:
            img = img

        return img

In [ ]:
aug_image = Aug_BengaliDataset(train0, img_height=HEIGHT, img_width=WIDTH, transform=transforms_train)


In [ ]:
nrow, ncol = 5, 2
fig, axs = plt.subplots(nrow, ncol, figsize=(15, 10))

for n in range(nrow):
    img = image[n]
    aug_img = aug_image[n]
    
    axs[n,0].imshow(img)
    axs[n,0].set_title(f'label: Original')
    axs[n,0].axis('off')
    
    axs[n,1].imshow(aug_img)
    axs[n,1].set_title(f'label: Augmix')
    axs[n,1].axis('off')
    
plt.tight_layout()
plt.show()